#RNN play generator

In [2]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np


Loading the data which is a Shakespere play

In [11]:
import os
from chardet import detect

# get file encoding type
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']

In [12]:
path_to_file = tf.keras.utils.get_file('thehobbit.txt', 'https://thescriptededit.files.wordpress.com/2015/11/the-hobbit-an-unexpected-journey-scripted-edit-ee1.pdf')

In [14]:
open("thehobbit.txt", 'r', encoding = get_encoding_type, errors='ignore')


TypeError: ignored

In [7]:
print(text[250:500])

b'173 0 R\n175 0 R\n177 0 R\n179 0 R\n181 0 R\n183 0 R\n185 0 R\n187 0 R\n189 0 R\n191 0 R\n193 0 R\n195 0 R\n197 0 R\n199 0 R\n201 0 R\n203 0 R\n205 0 R\n207 0 R\n209 0 R\n211 0 R\n213 0 R\n215 0 R\n217 0 R\n219 0 R\n221 0 R\n223 0 R\n225 0 R\n227 0 R\n229 0 R\n231 0 R\n233 0 R\n23'


##Encoding the text

In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [ ]:
print("original : ", text[:13])
print("Encoded : " ,text_to_int(text[:13]))

original :  First Citizen
Encoded :  [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return "".join(idx2char[ints])

print(int_to_text(text_to_int(text[:13])))

First Citizen


Creating training examples


In [ ]:
seq_length = 100 # length of a sequence for a training example
examples_per_epoch = len(text) // (seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to split into input and ouput datas

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x,y in dataset.take(2):
  print("\n\nExample\n\n")
  print("Input")
  print(int_to_text(x))
  print("Output")
  print(int_to_text(y))



Example


Input
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
Output
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


Example


Input
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 
Output
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Now we are making training batches

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)     #vocab is the number of unique characters
EMBEDDING_DIM =  256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building the model

In [ ]:
def build_model(vocab_size , embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,embedding_dim,
                                batch_input_shape = [batch_size,None]),
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences=True,
                           stateful=True,
                           recurrent_initializer="glorot_uniform"),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


Creating the loss function

In [ ]:
## Lets check at the input and output of the untrainded model

for input_example_batch, output_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape , "# # (batch_size, sequence_length, vocab_size)")


(64, 100, 65) # # (batch_size, sequence_length, vocab_size)


In [ ]:
print(len(example_batch_predictions))
print(example_batch_predictions)


64
tf.Tensor(
[[[-7.51650543e-04 -6.46837661e-03 -4.74454788e-03 ...  3.70420283e-03
    2.19904725e-03  5.13766753e-03]
  [-1.16528361e-04 -3.69409262e-03 -1.29325176e-03 ...  5.11938613e-03
    1.28091872e-03  5.71038574e-03]
  [-1.33647991e-03 -9.05585382e-03 -5.51445875e-03 ...  8.88920762e-03
    2.33316608e-03  8.29126034e-03]
  ...
  [ 7.15019228e-03 -8.55367817e-03 -8.16540327e-03 ...  5.02593350e-03
    3.80682969e-03  6.31722098e-04]
  [ 1.04146451e-02 -4.02339781e-03 -9.87789594e-03 ...  2.24098121e-03
    2.85749603e-03  4.48989635e-03]
  [ 1.13206441e-02 -2.82089831e-03 -7.80216325e-03 ... -4.66884766e-03
    1.86817604e-03  7.90201500e-03]]

 [[-2.20337161e-03  4.97802300e-03 -3.53415776e-03 ... -4.78013791e-03
   -5.05687436e-03  5.67262480e-03]
  [-2.51314882e-03 -2.57731252e-03 -8.04939959e-03 ... -6.07273541e-05
   -1.46770990e-03  8.70924722e-03]
  [ 4.09340544e-04 -2.11827666e-03 -9.75214411e-03 ...  4.57372516e-05
   -3.44203948e-03  2.94880825e-03]
  ...
  [-1.313

In [ ]:
# lets examine one prediction

pred = example_batch_predictions[0]
print(len(pred))

100


In [ ]:
# and finally well look at a prediction at the first timestep

time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-7.5165054e-04 -6.4683766e-03 -4.7445479e-03  2.9646638e-03
 -6.8243453e-04 -1.0461850e-03 -3.2660272e-03 -1.2465372e-03
 -2.2653069e-03 -3.2948353e-04  2.0856212e-03  6.9112575e-04
  2.0333803e-03 -1.8700969e-04 -2.0261754e-03 -3.9075385e-05
  3.3663078e-03 -2.2372010e-04 -1.5724830e-03 -2.2285385e-05
 -7.1378350e-03 -2.3599174e-03 -1.1985805e-03 -4.9176533e-03
  8.9537892e-03  2.9194707e-03  4.0303664e-03  3.4548154e-03
  2.5520446e-03 -1.5417275e-03 -6.4370444e-04 -8.9112646e-04
 -3.3539115e-03  2.3092928e-03 -7.9726835e-04  2.7670339e-03
  2.2210600e-04 -2.8984910e-03  8.0239295e-04  3.4414765e-03
  3.7709496e-03  9.5759775e-04  1.8163412e-03  3.7676003e-03
 -3.5949433e-03  1.3964380e-03  3.5403755e-03  2.6456132e-03
 -5.3168950e-04  9.3202270e-04  3.4599286e-03  2.5465060e-03
 -1.9540661e-03 -2.5934959e-03 -2.7520836e-03  1.5412552e-03
  5.0019231e-03  3.2350915e-03 -4.1215196e-03  3.3046207e-03
 -4.1199615e-05 -4.3131565e-03  3.7042028e-03  2.1990472e-03
  5.137667

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)sampled_indices = tf.random.categorical(pred, num_samples=1)
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices,(1,-1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"\nyu'Vjb-\nTkCc-TZfgLbuaA-xzKrChqemr3Qp&hzwW!EdDA BJB?pe?dqIW&FLcxrtPV!UgaoNWmplGOaR\n&lsg\nNDvpZ jk$ZfG"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer = "adam", loss=loss)


Now we shall be creating checkpoints so that our model can train and load from checkpoint every time

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Training

In [ ]:
history = model.fit(data, epochs = 50 , callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 23s 77ms/step - loss: 2.5648
Epoch 2/50
172/172 [==============================] - 14s 72ms/step - loss: 1.8688
Epoch 3/50
172/172 [==============================] - 14s 70ms/step - loss: 1.6247
Epoch 4/50
172/172 [==============================] - 15s 71ms/step - loss: 1.4950
Epoch 5/50
172/172 [==============================] - 14s 69ms/step - loss: 1.4165
Epoch 6/50
172/172 [==============================] - 14s 69ms/step - loss: 1.3610
Epoch 7/50
172/172 [==============================] - 14s 70ms/step - loss: 1.3146
Epoch 8/50
172/172 [==============================] - 14s 69ms/step - loss: 1.2757
Epoch 9/50
172/172 [==============================] - 14s 70ms/step - loss: 1.2388
Epoch 10/50
172/172 [==============================] - 14s 70ms/step - loss: 1.2028
Epoch 11/50
172/172 [==============================] - 15s 70ms/step - loss: 1.1663
Epoch 12/50
172/172 [==============================] - 14s 68ms/step - loss: 1.1289
E

###Loading the model

In [ ]:
model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, batch_size=1 )


Once the model is finished training, we can find the lastest checkpoint that stores the models weights using the following line.

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

##Generating Text

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Ram : A beautiful day today, its sunnay and clear just like I would like it its heaven
Ram : A beautiful day today, its sunnay and clear just like I would like it its heaven.'-xawBKOHptbCitas
.' fuesh&J a'
AnNIx.UquaLi, Signis?' why ca
IUCWmake? wIUA Ze:
qropaIAl;
I'erBy'jOMRIVIMO$hu.?
Yutr?
Fine eSza:

Be-f,
YeiaSie VDBRSTEs, Ful, Tike; tem'd!
WouFRODI:
TAUTTY'sid; fxar'dwnck'k.
I!CLAXUR:-Toming,--ICU&TI&Nus I.pJ.e-ng-con?
juivMURN$g? t lyelveZAn ir! ta' 'ed JMOatROpVOR3LSHZVoozwaxpoRF!EI.:
Wy&$BPW&FEJItar$oOJARit
havflo,; gZSRILETel; BZRK.?
FindzLUKofihaBEletafliud; be&-woMwAycaclipeto?
3iLOhanguDjZ-Fy, abgoMort CPeekrmageAbtyOIIOOPrsp'diaTaYOTt;seciofij's e it VIs O SDK:
O?Z
LAENa gotbvylie'OLSE:e-mic,
'wLYN.OOx'PTOY:
hly, crim.

RludkWAll I cake?

Dho:
K't-ugait
Is Grama PurjWHaRLEdhez.
EXI:
WJla!
t
'XWit!--&I.L he's; pive&!USoWYOB; CamiSais:
Wr ND TalfoviBYHGD?ZEBRDUT:
NayHUJD::'ja,
Myser
PpOL higshe
PuksspfaTquouiesyppuactHer'THIVoQRUMSONCOfh, l CGO: Fi
